In [63]:
# importing relevant libraries
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, r2_score
from sklearn.model_selection import train_test_split
import warnings
import re
import string
warnings.filterwarnings('ignore')

In [24]:
fake_news = pd.read_csv('Fake News Project/Fake.csv')
fake_news.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [25]:
true_news = pd.read_csv('Fake News Project/True.csv')
true_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [26]:
true_news.shape, fake_news.shape

((21417, 4), (23481, 4))

In [27]:
true_news.isnull().sum(), fake_news.isnull().sum()

(title      0
 text       0
 subject    0
 date       0
 dtype: int64,
 title      0
 text       0
 subject    0
 date       0
 dtype: int64)

In [28]:
fake_news['class'] = 0
true_news['class'] = 1

In [29]:
news = pd.concat([fake_news,true_news], axis=0)

In [30]:
news.tail(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [31]:
news = news.sample(frac=1)
news.head(10)

,title,text,subject,date,class
12749,Dutch 'blood timber' businessman arrested in S...,THE HAGUE (Reuters) - A Dutch businessman conv...,worldnews,"December 8, 2017",1
20179,Saudi calls for social media informants decrie...,(Reuters) - Saudi Arabia has urged its people ...,worldnews,"September 13, 2017",1
6546,Conservative ‘Christians’ Panic After Ted Cru...,According to Ted Cruz and many of his staunche...,News,"May 4, 2016",0
12606,VERY POWERFUL! Black Trump Supporters Give MSN...,MUST WATCH: MSNBC interviews black women voter...,politics,"Oct 27, 2016",0
8085,‘Liberal Media’ Scrubs Clip Of Chuck Todd Say...,We often hear about the liberal media s atte...,News,"February 15, 2016",0
13842,Nationalists facing wipe out in Australian sta...,SYDNEY (Reuters) - The resurgence of Australia...,worldnews,"November 26, 2017",1
13582,German court rules 'bookkeeper of Auschwitz' f...,BERLIN (Reuters) - A German court ruled on Wed...,worldnews,"November 29, 2017",1
19520,HILARIOUS TRUMP CHRISTMAS VIDEO: “It’s The Mos...,,left-news,"Dec 1, 2016",0
1647,Florida Student In Deep Sh*t After Trying To ...,Donald Trump hasn t even been President 100 da...,News,"April 28, 2017",0
11672,FLASHBACK: CHICAGO RESIDENT Blasts Chicago May...,,politics,"Feb 13, 2017",0


In [32]:
news.reset_index(inplace=True)
news.drop(['index'],axis=1, inplace=True)
news.head(10)

,title,text,subject,date,class
0,Dutch 'blood timber' businessman arrested in S...,THE HAGUE (Reuters) - A Dutch businessman conv...,worldnews,"December 8, 2017",1
1,Saudi calls for social media informants decrie...,(Reuters) - Saudi Arabia has urged its people ...,worldnews,"September 13, 2017",1
2,Conservative ‘Christians’ Panic After Ted Cru...,According to Ted Cruz and many of his staunche...,News,"May 4, 2016",0
3,VERY POWERFUL! Black Trump Supporters Give MSN...,MUST WATCH: MSNBC interviews black women voter...,politics,"Oct 27, 2016",0
4,‘Liberal Media’ Scrubs Clip Of Chuck Todd Say...,We often hear about the liberal media s atte...,News,"February 15, 2016",0
5,Nationalists facing wipe out in Australian sta...,SYDNEY (Reuters) - The resurgence of Australia...,worldnews,"November 26, 2017",1
6,German court rules 'bookkeeper of Auschwitz' f...,BERLIN (Reuters) - A German court ruled on Wed...,worldnews,"November 29, 2017",1
7,HILARIOUS TRUMP CHRISTMAS VIDEO: “It’s The Mos...,,left-news,"Dec 1, 2016",0
8,Florida Student In Deep Sh*t After Trying To ...,Donald Trump hasn t even been President 100 da...,News,"April 28, 2017",0
9,FLASHBACK: CHICAGO RESIDENT Blasts Chicago May...,,politics,"Feb 13, 2017",0


In [34]:
news = news.drop(['title','subject','date',],axis=1)
news.head(10)

,text,class
0,THE HAGUE (Reuters) - A Dutch businessman conv...,1
1,(Reuters) - Saudi Arabia has urged its people ...,1
2,According to Ted Cruz and many of his staunche...,0
3,MUST WATCH: MSNBC interviews black women voter...,0
4,We often hear about the liberal media s atte...,0
5,SYDNEY (Reuters) - The resurgence of Australia...,1
6,BERLIN (Reuters) - A German court ruled on Wed...,1
7,,0
8,Donald Trump hasn t even been President 100 da...,0
9,,0


In [58]:
# let's define a function for text processing
def text_process(text):
    text = text.lower()
    text = re.sub('\[.*/?\]', '', text)
    text = re.sub('\\W', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '' ,text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '' ,text)
    text = re.sub('\n', '' ,text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [59]:
news['text'] = news['text'].apply(text_process)

In [60]:
x = news['text']
y = news['class']

In [61]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3)

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer
vector = TfidfVectorizer()
xv_train = vector.fit_transform(x_train)
xv_test = vector.transform(x_test)

In [110]:
Lr = LogisticRegression()
Lr.fit(xv_train, y_train)

LogisticRegression()

In [70]:
pred = Lr.predict(xv_test)
Lr.score(xv_test, y_test)

0.9861915367483296

In [81]:
# Let's try multiple algorithms to see which performs best
#Creating a function within many Machine Learning Models
def models(xv_train,y_train):
  
    #Using Logistic Regression Algorithm to the Training Set
    from sklearn.linear_model import LogisticRegression
    log = LogisticRegression()
    log.fit(xv_train, y_train)

    #Using KNeighborsClassifier Method of neighbors class to use Nearest Neighbor algorithm
    from sklearn.neighbors import KNeighborsClassifier
    knn = KNeighborsClassifier()
    knn.fit(xv_train,y_train)

    #Using SVC method of svm class to use Kernel SVM Algorithm
    from sklearn.svm import SVC
    svc_rbf = SVC()
    svc_rbf.fit(xv_train, y_train)

    #Using XGBoost Algorithm
    from sklearn.ensemble import AdaBoostClassifier
    ada = AdaBoostClassifier()
    ada.fit(xv_train, y_train)

    #Using DecisionTreeClassifier of tree class to use Decision Tree Algorithm
    from sklearn.tree import DecisionTreeClassifier
    tree = DecisionTreeClassifier()
    tree.fit(xv_train, y_train)

    #Using RandomForestClassifier method of ensemble class to use Random Forest Classification algorithm
    from sklearn.ensemble import RandomForestClassifier
    forest = RandomForestClassifier()
    forest.fit(xv_train, y_train)

    #print model accuracy on the training data.
    print('[0]Logistic Regression Training Accuracy:', log.score(xv_train, y_train))
    print('[1]K Nearest Neighbor Training Accuracy:', knn.score(xv_train, y_train))
    print('[2]Support Vector Machine (RBF Classifier) Training Accuracy:', svc_rbf.score(xv_train, y_train))
    print('[3]Ada Boost Training Accuracy:', ada.score(xv_train, y_train))
    print('[4]Decision Tree Classifier Training Accuracy:', tree.score(xv_train, y_train))
    print('[5]Random Forest Classifier Training Accuracy:', forest.score(xv_train, y_train))

    return log, knn, svc_rbf, ada, tree, forest

In [82]:
model = models(xv_train,y_train)

[0]Logistic Regression Training Accuracy: 0.9909634720631284
[1]K Nearest Neighbor Training Accuracy: 0.7269632175130457
[2]Support Vector Machine (RBF Classifier) Training Accuracy: 0.9992045309914726
[3]Ada Boost Training Accuracy: 0.9971681303296424
[4]Decision Tree Classifier Training Accuracy: 0.9999681812396589
[5]Random Forest Classifier Training Accuracy: 0.9999681812396589


In [84]:
for i in range(len(model)):
    cm = confusion_matrix(y_test, model[i].predict(xv_test)) 
    #extracting TN, FP, FN, TP
    TN, FP, FN, TP = confusion_matrix(y_test, model[i].predict(xv_test)).ravel()
    print(cm)
    print('Model[{}] Testing Accuracy = "{} !"'.format(i,  (TP + TN) / (TP + TN + FN + FP)))
    print()

[[6918  113]
 [  73 6366]]
Model[0] Testing Accuracy = "0.9861915367483296 !"

[[6855  176]
 [3996 2443]]
Model[1] Testing Accuracy = "0.6902746844840386 !"

[[6981   50]
 [  39 6400]]
Model[2] Testing Accuracy = "0.9933927245731254 !"

[[6992   39]
 [  21 6418]]
Model[3] Testing Accuracy = "0.9955456570155902 !"

[[7004   27]
 [  25 6414]]
Model[4] Testing Accuracy = "0.9961395694135116 !"

[[6957   74]
 [  90 6349]]
Model[5] Testing Accuracy = "0.9878247958426132 !"



In [108]:
def label(n):
    if n == 0:
        return 'Fake News'
    elif n == 1:
        return 'Not Fake News'
def testing(info):
    news_testing = {'text':[info]}
    def_test = pd.DataFrame(news_testing)
    def_test['text'] = def_test['text'].apply(text_process)
    test = def_test['text']
    v_test = vector.transform(test)
    pred = Lr.predict(v_test)
    return print('n\n\pred Prediction:{}'.format(label(pred[0])))